## Rules based checks

You can use rules and simple functions both for filtering and deduplication of the data and for checking the quality of synthetic data generations. 

As you'd expect the rules that you define will be based on the domain and the data that you have.

In [2]:
dataset[0]['generations']

['This dataset contains categorized records of instructions, contexts, and responses, curated to ensure clarity, accuracy, and relevance, suitable for training AI models to understand and respond to instructions effectively.',
 'The Databricks-Dolly-15k-Curated dataset comprises corrected records of instructions, context, and responses that adhere to specific categories, aiming to ensure high-quality data and real information, which can be utilized to enhance natural language understanding and generation tasks.']

## Checking the quality of synthetic data

The dataset below is a synthetic dataset generated which is aiming to use an LLM to generate tl;dr summaries of dataset cards. We have some instruction in our prompt which tell the model to not repeat information that is probably better as structured metadata. In the synthetic dataset generation pipeline, we use a judge LLM to check the quality of the generated summaries against the instructions. 

In [15]:
from datasets import load_dataset

dataset = load_dataset(
    "davanstrien/dataset-preferences-llm-course-full-dataset", split="train"
)

In [16]:
dataset

Dataset({
    features: ['datasetId', 'card', 'instruction', 'system_prompt', 'generation_models', 'generations', 'model_name', 'ratings', 'rationales'],
    num_rows: 2482
})

In [17]:
dataset[0]['rationales']

["The text concisely summarizes the dataset, focusing on its critical aspects, and meets all the restrictions. It accurately describes the dataset's content and potential use, without speculating or mentioning unnecessary information.",
 'The text partially complies with the instruction. It summarizes the dataset\'s content and goal, but mentions "high-quality data," which is not explicitly mentioned in the dataset card and goes against the restriction of avoiding references to the quality of the dataset. Additionally, it\'s a bit wordy and could be more concise.']

One of the instructions we give the LLM is not to give quality judgements about the dataset in the card i.e. don't say things like "this is a high quality dataset". Although the judge LLM will identify some of these we can also do a much cheaper and simpler check by using a rules based approach.

In [37]:
def check_is_high_quality(row):
    generations = row['generations']
    # check for Nones
    if any(generation is None for generation in generations):
        return False
    return any("high quality" or "high-quality" in generation for generation in generations)


In [39]:
dataset.filter(check_is_high_quality)[0]

{'datasetId': 'argilla/databricks-dolly-15k-curated-en',
 'card': "In this dataset, you will find a collection of records that show a category, an instruction, a context and a response to that instruction. The aim of the project is to correct the instructions, intput and responses to make sure they are of the highest quality and that they match the task category that they belong to. All three texts should be clear and include real information. In addition, the response should be as complete but concise as possible.\nTo curate the dataset, you will need to provide an answer to the following text fields:\n1 - Final instruction:\nThe final version of the instruction field. You may copy it using the copy icon in the instruction field. Leave it as it is if it's ok or apply any necessary corrections. Remember to change the instruction if it doesn't represent well the task category of the record.\n2 - Final context:\nThe final version of the instruction field. You may copy it using the copy i

We also don't want the generated summaries to mention the licence of the dataset as this is already in the structured metadata.

In [40]:
for rationales in dataset[0]['rationales']:
    for rationale in rationales:
        if "license" in rationale:
            print(rationale)

In [41]:
import re

def check_license_text(text):
    # List of regex patterns for common software licenses and date formats
    license_patterns = [
        r'\bMIT License\b',
        r'\bGNU General Public License\b',
        r'\bGPL\b',
        r'\bApache License\b',
        r'\bBSD License\b',
        r'\bMozilla Public License\b',
        r'\bMPL\b',
        r'\bCreative Commons\b',
        r'\bCC-BY\b',
        r'\bCC-BY-SA\b',
        r'\bProprietary License\b',
        # Common date formats
        r'\b\d{4}-\d{2}-\d{2}\b',  # YYYY-MM-DD
        r'\b\d{4}/\d{2}/\d{2}\b',  # YYYY/MM/DD
        r'\b\d{2}/\d{2}/\d{4}\b',  # MM/DD/YYYY
        r'\b\d{2}/\d{2}/\d{4}\b',  # DD/MM/YYYY
        r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December) \d{1,2}, \d{4}\b'
    ]

    # Compile patterns into a single regex
    combined_pattern = re.compile('|'.join(license_patterns), re.IGNORECASE)

    # Check if any of the patterns match a substring within the text
    if combined_pattern.search(text):
        return True
    else:
        return False

# Example usage
texts = [
    "This software is licensed under the MIT License.",
    "Released under the GNU General Public License, version 3.",
    "This code follows the Apache License 2.0 guidelines.",
    "All rights reserved under a Proprietary License.",
    "The project is licensed under Creative Commons CC-BY-SA 4.0.",
    "The document was last updated on 2023-05-14.",
    "This agreement was signed on January 1, 2024.",
    "No relevant license or date information here."
]

# Testing the function
results = {text: check_license_text(text) for text in texts}
print(results)

{'This software is licensed under the MIT License.': True, 'Released under the GNU General Public License, version 3.': True, 'This code follows the Apache License 2.0 guidelines.': True, 'All rights reserved under a Proprietary License.': True, 'The project is licensed under Creative Commons CC-BY-SA 4.0.': True, 'The document was last updated on 2023-05-14.': True, 'This agreement was signed on January 1, 2024.': True, 'No relevant license or date information here.': False}


We can also filter on things like mentions of train and test splits, as this is not relevant to the summary of the dataset card and is something we prompt the model to not include.

In [10]:
import re

def check_train_test_val_splits(text):
    """
    Function to check for mentions of train, test, and validation splits in the provided text.

    Parameters:
    text (str): The input text to check.

    Returns:
    dict: A dictionary indicating whether train, test, and validation splits are mentioned.
    """

    # Define regex patterns for train, test, and validation splits
    patterns = {
        "train": re.compile(r'\btrain(?:ing)?(?:\s*set)?(?:\s*split)?\b', re.IGNORECASE),
        "test": re.compile(r'\btest(?:ing)?(?:\s*set)?(?:\s*split)?\b', re.IGNORECASE),
        "validation": re.compile(r'\b(?:validation|val|dev)(?:\s*set)?(?:\s*split)?\b', re.IGNORECASE),
        "numbers": re.compile(r'\b(?:\d+|\d*\.\d+)(?:\s*k|K|%|\s*percent)?\b', re.IGNORECASE)
    }

    # Initialize results dictionary
    results = {
        "train": False,
        "test": False,
        "validation": False,
        "numbers": []
    }

    # Check for matches in the text
    for key, pattern in patterns.items():
        if matches := pattern.findall(text):
            if key == "numbers":
                results[key].extend(matches)
            else:
                results[key] = True

    return results

# Example usage
text = """
The dataset is split into a training set, a test set, and a validation set.
"""

result = check_train_test_val_splits(text)
print(result)

{'train': True, 'test': True, 'validation': True, 'numbers': []}


## SQL rules

In [42]:
sql_dataset = load_dataset("gretelai/synthetic_text_to_sql", split="train")

In [43]:
sql_dataset

Dataset({
    features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
    num_rows: 100000
})

In [47]:
sql_dataset[:4]['sql_prompt']

['What is the total volume of timber sold by each salesperson, sorted by salesperson?',
 'List all the unique equipment types and their corresponding total maintenance frequency from the equipment_maintenance table.',
 'How many marine species are found in the Southern Ocean?',
 'What is the total trade value and average price for each trader and stock in the trade_history table?']

# Preventing certain kinds of input

We may want to not allow users to add SQL to their prompts. This is something we can add at the application layer but we might also want to exclude this kind of prompt form our training data, especially since the model may appear to be doing better than it is if it is just copying the SQL from the prompt (and this won't be available in production).

In [51]:
sql_dataset.filter(lambda x: "SELECT" in x['sql_prompt'])[:]['sql_prompt']

['SELECT MemberID, Name, Age, Gender, AVG(WorkoutDuration) as AverageWorkoutDuration FROM Members JOIN Workouts ON Members.MemberID = Workouts.MemberID GROUP BY MemberID, Name, Age, Gender ORDER BY AverageWorkoutDuration DESC;',
 'SELECT MemberID, AVG(Steps) as AverageSteps, AVG(Calories) as AverageCalories, AVG(HeartRate) as AverageHeartRate FROM Wearables GROUP BY MemberID ORDER BY AverageSteps DESC;',
 "SELECT MemberID, COUNT(*) as WorkoutCountThisWeek FROM Workouts WHERE Date >= DATE_TRUNC('week', CURRENT_DATE) GROUP BY MemberID ORDER BY WorkoutCountThisWeek DESC;",
 "SELECT DISTINCT MemberID, Gender FROM Members WHERE State = 'CA';",
 "SELECT MemberID, WorkoutType, DATE_TRUNC('day', Date) as Day, MIN(Duration) as MinDurationPerDay FROM Workouts GROUP BY MemberID, WorkoutType, Day ORDER BY Day DESC;",
 "SELECT MemberID, COUNT(*) as WorkoutCountThisMonth FROM Workouts WHERE DATE_TRUNC('month', Date) = DATE_TRUNC('month', CURRENT_DATE) GROUP BY MemberID ORDER BY WorkoutCountThisMonth

In [52]:
sql_dataset.filter(lambda x: "FROM" in x['sql_prompt'])[:4]['sql_prompt']

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

['SELECT MemberID, Name, Age, Gender, AVG(WorkoutDuration) as AverageWorkoutDuration FROM Members JOIN Workouts ON Members.MemberID = Workouts.MemberID GROUP BY MemberID, Name, Age, Gender ORDER BY AverageWorkoutDuration DESC;',
 'SELECT MemberID, AVG(Steps) as AverageSteps, AVG(Calories) as AverageCalories, AVG(HeartRate) as AverageHeartRate FROM Wearables GROUP BY MemberID ORDER BY AverageSteps DESC;',
 "SELECT MemberID, COUNT(*) as WorkoutCountThisWeek FROM Workouts WHERE Date >= DATE_TRUNC('week', CURRENT_DATE) GROUP BY MemberID ORDER BY WorkoutCountThisWeek DESC;",
 "SELECT DISTINCT MemberID, Gender FROM Members WHERE State = 'CA';"]

## Preventing certain kinds of output

Similarly we may want to prevent the model from violating certain rules in the SQL it generates. We can do a more "formal" check or compilation of the SQL to check for this but we can also do a simple check for certain keywords or phrases that we don't want to appear in the generated SQL.

![](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

In [53]:
sql_dataset.filter(lambda x: "DROP" in x['sql'])[:4]['sql']

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

['DROP TABLE vehicle_safety_testing;',
 'DROP TABLE redundant_billing_data;',
 'DROP TABLE conditions;',
 'DROP VIEW young_engineers;']

We can ask an LLM to help us identify things we might want to be checking for. In the past we might have been to lazy to write a lot of functions that will only capture very small amounts of rules but with the LLM we can do this much more easily.

![](claude_sql.png)

In [54]:
import re

def has_double_quoted_strings(sql):
    pattern = r'"[^"]*"'
    matches = re.findall(pattern, sql)
    return len(matches) > 0

In [55]:
sql_dataset.filter(lambda x: has_double_quoted_strings(x["sql"]))['sql']

['SELECT program_id FROM professional_development JOIN teachers t ON professional_development.attended_teacher_id = t.teacher_id GROUP BY program_id HAVING COUNT(DISTINCT t.teacher_id) > 1 AND t.state = "NY";',
 'SELECT COUNT(*) FROM students s1 WHERE s1.mental_health_score > (SELECT AVG(s2.mental_health_score) FROM students s2 WHERE s2.state = "CA");',
 'SELECT YEAR(completion_date) AS "Completion Year", COUNT(*) FROM economic_diversification WHERE project_status = \'completed\' GROUP BY YEAR(completion_date);',
 'SELECT MIN(listing_price) AS "Lowest Price", MAX(listing_price) AS "Highest Price" FROM sustainable_urban WHERE city = \'New York\';',
 'SELECT MIN(productivity) AS "Minimum productivity", MAX(productivity) AS "Maximum productivity" FROM productivity WHERE department = \'environment\' AND year = 2021;',
 'SELECT MIN(program_count) AS "Minimum community programs", MAX(program_count) AS "Maximum community programs" FROM community_programs WHERE country IN (\'Latin America\');'

In [11]:
def has_having_without_group_by(sql):
    pattern = r'HAVING\s+(?!.*\bGROUP\s+BY\b)'
    match = re.search(pattern, sql, re.IGNORECASE | re.DOTALL)
    return match is not None

In [13]:
def has_sum_without_null_handling(sql):
    pattern = r'SUM\s*\(\s*\w+\s*\)'
    match = re.search(pattern, sql, re.IGNORECASE)
    return match is not None

In [14]:
sql_dataset.filter(lambda x: has_sum_without_null_handling(x["sql"]))['sql']

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

['SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id GROUP BY salesperson_id, name ORDER BY total_volume DESC;',
 'SELECT equipment_type, SUM(maintenance_frequency) AS total_maintenance_frequency FROM equipment_maintenance GROUP BY equipment_type;',
 'SELECT SUM(spending) FROM defense.eu_humanitarian_assistance WHERE year BETWEEN 2019 AND 2021;',
 "SELECT SUM(fare) FROM bus_routes WHERE route_name = 'Green Line';",
 "SELECT SUM(Amount) AS TotalAssistance, Country FROM HumanitarianAssistance WHERE Organization NOT IN ('Government', 'Military') AND Year BETWEEN 2016 AND 2020 GROUP BY Country;",
 "SELECT SUM(value) FROM socially_responsible_loans WHERE client_gender = 'female' AND date BETWEEN '2021-07-01' AND '2021-12-31';",
 "SELECT SUM(data_usage) FROM mobile_usage WHERE city = 'Toronto' AND year = 2022;",
 'SELECT region, SUM(incidents) as total_incidents FROM incident_region WHERE inc